# RFSoC IQ Mixing

This notebook produces DAC pulses that are 90 degrees phase shifted for use with an external IQ mixer. DAC_A is channel I and DAC_B is channel Q. DAC_A therefore leads DAC_B by 90 degrees.

In [3]:
# Import libraries
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../rfsoc_lib')
from rfsoc_calibration import RfsocCalibration
from rfsoc_pulses import RfsocPulses

import pickle
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
# Board libraries
from qick import *
import Pyro4

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [5]:
# Test pulse sequence
imported_sequences = {"MW": [(5000, 1.0, 0.1, 0)]}

In [6]:
dac_delay = 0
dac_gain = 10000

# Assign imported pulse sequence to DAC_A and DAC_B, setting gain and delays
pulses = RfsocPulses(imported_sequences,
                              ch_map = {"DAC_A":"MW", "DAC_B":"MW"},
                              delays = {"DAC_A":dac_delay, "DAC_B":dac_delay},
                              gains  = {"DAC_A":dac_gain, "DAC_B":dac_gain},
                              iq_mix = True,
                              )

----- DAC A -----
ch_type: DAC
ch_index: 1
delay: 0
gain: 10000
lengths: [5.0]
times: [0.0]
amps: [1.0]
freqs: [100.0]
phases: [0.0]
num_pulses: 1
duration: 5.0
----- DAC B -----
ch_type: DAC
ch_index: 0
delay: 0
gain: 10000
lengths: [5.0]
times: [0.0]
amps: [1.0]
freqs: [100.0]
phases: [-90.0]
num_pulses: 1
duration: 5.0
----- End time: 5.0 -----


In [8]:
# Import phase calibration data
with open('../pickles/calibrate_phase.pickle', 'rb') as handle:
    dac_phis = pickle.load(handle)
calibration = RfsocCalibration(dac_phis)

In [19]:
# Program RFSoC with pulse sequence
prog = QickProgram(soccfg)
pulses.generate_asm(prog, dac_phis, reps=1)
pulses.config_internal_start(soc, prog)
soc.tproc.start()
print(prog)


// Program

        synci 200;
        regwi 0, $14, 0;
LOOP_I: regwi 1, $22, 43690667;                 //freq = 43690667
        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 10000;                    //gain = 10000
        regwi 1, $26, 592896;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 3072 
        regwi 1, $27, 0;                        //t = 0
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 0, $22, 43690667;                 //freq = 43690667
        regwi 0, $23, 908379756;                //phase = 3633519027
        bitwi 0, $23, $23 << 2;
        mathi 0, $23, $23 + 3;
        regwi 0, $25, 10000;                    //gain = 10000
        regwi 0, $26, 592896;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 3072 
        regwi 0, $27, 0;                        //t = 0
        set 0, 0, $22, $23, $0, $25, $26, $27;  //ch = 0, pulse @t = $27
        waiti 0

In [ ]:
# Simple implementation where DACs continuously output a fixed frequency

# # Config parameters
# freq = 100
# phases = [-90, 0]
# gain = 10000

# # Program DAC registers
# prog = QickProgram(soccfg)
# soc.reset_gens()
# for ch in range(2):
#     prog.declare_gen(ch=ch, nqz=1)
#     prog.set_pulse_registers(ch = ch,
#                              gain = gain,
#                              freq = prog.freq2reg(freq, gen_ch=ch),
#                              phase = prog.deg2reg(phases[ch]+calibration.phase(freq, ch), gen_ch=ch),
#                              style = "const",
#                              length = prog.us2cycles(100, gen_ch=ch),
#                              mode = "periodic",
#                             )
#     prog.pulse(ch=ch, t=0)

# # Start DACs
# prog.config_all(soc)
# soc.tproc.start()

In [ ]:
# Stop DACs from generating waveforms
soc.reset_gens()